<a href="https://colab.research.google.com/github/Tolga-dev/DataVisulation/blob/main/DataVisulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Migration Data Visualization Notebook

This notebook is designed to explore and answer various migration-related questions using interactive visualizations. The primary goal is to visualize and analyze immigration, emigration, and refugee data year by year, from 1995 to today. Below are the key questions this notebook aims to answer:

## Key Questions:
1. **Which country has the highest immigration from 1995 to today?**
2. **Which country has the highest emigration from 1995 to today?**
3. **What are the year-by-year refugee statistics by country of origin from 1995 to today?**
4. **How has immigration evolved year by year?**
5. **How has emigration evolved year by year?**
6. **How have refugee statistics evolved year by year, by country of origin?**

The notebook provides interactive visualizations such as choropleth maps and bar charts to answer these questions.

In [47]:
# Check and install necessary packages if not available
try:
    import plotly.express as px
except ImportError:
    !pip install plotly

try:
    import pandas as pd
except ImportError:
    !pip install pandas

try:
    import json
except ImportError:
    !pip install json

try:
    import dash
except ImportError:
    !pip install dash

try:
    from dash import dcc, html
    from dash.dash_table import DataTable
    from dash.dependencies import Input, Output
except ImportError:
    !pip install dash

try:
    from google.colab import drive
except ImportError:
    !pip install google-colab

try:
    import requests
except ImportError:
    !pip install requests

try:
    import gdown
except ImportError:
    !pip install gdown


## Installation of Required Packages

This section of the code ensures that all necessary Python libraries are installed in the environment. It checks if each package (e.g., `plotly`, `pandas`, `dash`, `requests`, etc.) is already imported. If any package is missing, it uses `!pip install` to install the required library

In [48]:
# Links to the files in Google Drive (Replace these with your actual links)
refugees = "https://drive.google.com/uc?id=1SHwts3iNlZWj6lnP-pgMuXMPaAAZX7S9"
countryJson = "https://drive.google.com/uc?id=1N5DNSrLHrJhroVCvw1kt2e77EeDS_T_q"
emigrant = "https://drive.google.com/uc?id=1E5Jy3_QMiBhYJ3V4KiCt6khydgaGwxpo"
immegrant = "https://drive.google.com/uc?id=1ZuWnfvI8K6PGM9YfGTS2Loiwyviu6JY4"

# Download GeoJSON file (for visualization)
gdown.download(countryJson, "countries.geojson", quiet=False)

# Load the GeoJSON
with open("countries.geojson") as f:
    geojson = json.load(f)

# Download CSV files
gdown.download(emigrant, "total-number-of-emigrants.csv", quiet=False)
gdown.download(immegrant, "total-number-of-international-immigrants.csv", quiet=False)
gdown.download(refugees, "refugees-by-country-of-origin.csv", quiet=False)

# Load the CSV data (assuming it has columns: 'Entity', 'Year', 'Total number of emigrants' and 'Total number of international immigrants')
df_emigrants = pd.read_csv("total-number-of-emigrants.csv")
df_immigrants = pd.read_csv("total-number-of-international-immigrants.csv")
df_refugees_by_country = pd.read_csv("refugees-by-country-of-origin.csv")

# Print the first few rows to verify
print(df_emigrants.head())
print(df_immigrants.head())
print(df_refugees_by_country.head())

Downloading...
From: https://drive.google.com/uc?id=1N5DNSrLHrJhroVCvw1kt2e77EeDS_T_q
To: /content/countries.geojson
100%|██████████| 24.3M/24.3M [00:00<00:00, 123MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1E5Jy3_QMiBhYJ3V4KiCt6khydgaGwxpo
To: /content/total-number-of-emigrants.csv
100%|██████████| 47.4k/47.4k [00:00<00:00, 40.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZuWnfvI8K6PGM9YfGTS2Loiwyviu6JY4
To: /content/total-number-of-international-immigrants.csv
100%|██████████| 46.0k/46.0k [00:00<00:00, 40.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SHwts3iNlZWj6lnP-pgMuXMPaAAZX7S9
To: /content/refugees-by-country-of-origin.csv
100%|██████████| 137k/137k [00:00<00:00, 21.2MB/s]

        Entity  Year  Total number of emigrants
0  Afghanistan  1990                    7679582
1  Afghanistan  1995                    4347049
2  Afghanistan  2000                    4750677
3  Afghanistan  2005                    4116739
4  Afghanistan  2010                    5269518
        Entity  Year  Total number of international immigrants
0  Afghanistan  1990                                     57686
1  Afghanistan  1995                                     71522
2  Afghanistan  2000                                     75917
3  Afghanistan  2005                                     87314
4  Afghanistan  2010                                    102276
        Entity  Year  Refugees by country of origin
0  Afghanistan  1979                         500000
1  Afghanistan  1980                        1734921
2  Afghanistan  1981                        3879984
3  Afghanistan  1982                        4488214
4  Afghanistan  1983                        4712735


## Downloading Data Files from Google Drive

This section of the code is used to download the necessary data files from Google Drive and load them for further analysis. The links provided should be replaced with your actual Google Drive file links.

1. **Google Drive Links**:
   - `refugees`: Link to the CSV file containing refugee data.
   - `countryJson`: Link to the GeoJSON file containing country shapes for visualization.
   - `emigrant`: Link to the CSV file containing data about emigrants.
   - `immegrant`: Link to the CSV file containing data about immigrants.

2. **Downloading Files**:
   - `gdown.download()` is used to download the files from the provided Google Drive links and save them locally in the working directory.

3. **Loading Data**:
   - The downloaded GeoJSON file is opened and loaded using the `json.load()` function to make it available for visualization.
   - CSV files are loaded into Pandas DataFrames using `pd.read_csv()`. The DataFrames contain columns such as 'Entity', 'Year', 'Total number of emigrants', and 'Total number of international immigrants'

In [49]:

years_of_interest = list(range(1990, 2025, 5))

# Filter the dataframe for the years of interest
df_filtered = df_refugees_by_country[df_refugees_by_country["Year"].isin(years_of_interest)]

# Convert the filtered data to a dictionary
# Format: {'Country': {1990: value, 1995: value, ...}}
refugees_dict = {}
for _, row in df_filtered.iterrows():
    country = row["Entity"]
    year = row["Year"]
    value = row["Refugees by country of origin"]

    if country not in refugees_dict:
        refugees_dict[country] = {}
    refugees_dict[country][year] = value


# Filter relevant columns and rows with valid data
df_emigrants = df_emigrants[['Entity', 'Year', 'Total number of emigrants']].dropna()
df_immigrants = df_immigrants[['Entity', 'Year', 'Total number of international immigrants']].dropna()

# Match data to GeoJSON countries
country_names_geojson = [feature['properties']['ADMIN'] for feature in geojson['features']]
df_emigrants = df_emigrants[df_emigrants['Entity'].isin(country_names_geojson)]
df_immigrants = df_immigrants[df_immigrants['Entity'].isin(country_names_geojson)]

# Initialize the main data structure
country_data = {}
years_data = {}
current_country = df_emigrants['Entity'].iloc[0]  # Start with the first country

# Process emigrant and immigrant data
for _, row in df_emigrants.iterrows():
    country = row['Entity']
    year = int(row['Year'])
    total_emigrants = float(row['Total number of emigrants'])

    # Match corresponding immigrant data
    immigrant_row = df_immigrants[(df_immigrants['Entity'] == country) & (df_immigrants['Year'] == year)]
    total_immigrants = float(immigrant_row['Total number of international immigrants'].iloc[0]) if not immigrant_row.empty else 0

    # Add to country data
    if country != current_country:
        country_data[current_country] = years_data
        current_country = country
        years_data = {}

    # Calculate net migration
    years_data[year] = {
        'emigrants': total_emigrants,
        'immigrants': total_immigrants,
        'net-migration': total_immigrants - total_emigrants  # Net Migration
    }

# Add the last country's data
if years_data:
    country_data[current_country] = years_data

# Add refugee data to country_data
for country, yearly_data in refugees_dict.items():
    if country in country_data:  # If the country exists in the main dataset
        for year, refugees in yearly_data.items():
            if year in country_data[country]:  # If the year exists for this country
                # Add refugee data
                country_data[country][year]['refugees'] = refugees
            else:
                # Create a new entry for the year
                country_data[country][year] = {
                    'emigrants': 0,
                    'immigrants': 0,
                    'net-migration': 0,
                    'refugees': refugees
                }
    else:
        # Create a new entry for the country
        country_data[country] = {
            year: {
                'emigrants': 0,
                'immigrants': 0,
                'net-migration': 0,
                'refugees': refugees
            }
            for year, refugees in yearly_data.items()
        }




## Data Processing for Migration, Refugee, Emigrant, and Immigrant Data

This section of the code processes the data for emigrants, immigrants, and refugees, filters it for the years of interest, and matches it to GeoJSON data for visualization. The data is then organized into a structure that allows for efficient analysis and visualization.

### Steps:

1. **Defining Years of Interest**:
   - The code defines a list `years_of_interest` with years ranging from 1990 to 2025 in 5-year intervals (1990, 1995, 2000, etc.).

2. **Filtering Refugee Data**:
   - The refugee data is filtered to include only the rows for the specified years of interest. It then organizes the filtered data into a dictionary, `refugees_dict`, where each country has its own nested dictionary with years as keys and refugee counts as values.

3. **Cleaning and Filtering Emigrant and Immigrant Data**:
   - The data for emigrants and immigrants is filtered to include only the relevant columns ('Entity', 'Year', and the respective migration counts). Rows with missing values are removed using `.dropna()`.
   - The data is then matched with the country names in the GeoJSON file to ensure that only countries present in both the dataset and the GeoJSON file are included.

4. **Processing Data for Each Country**:
   - The main data structure, `country_data`, is initialized, which will store data for each country across the years of interest.
   - For each country in the emigrant dataset, the corresponding immigrant data is matched. The net migration for each year is calculated by subtracting the number of emigrants from immigrants.
   - The data for each country is added to `country_data` with each year containing emigrant, immigrant, and net migration counts.

5. **Adding Refugee Data**:
   - The refugee data (`refugees_dict`) is added to the `country_data` structure. For each country and year combination, the number of refugees is added to the data. If the country and year do not already exist in the dataset, new entries are created with default values for emigrants, immigrants, and net migration.

6. **Final Data Structure**:
   - The final `country_data` structure contains migration and refugee data for each country across the years of interest, allowing for comprehensive analysis and visualization.


In [50]:


app = dash.Dash(__name__)

# Layout with two Sliders
slider_emigrants = dcc.Slider(
    id='year-slider-emigrants',
    min=1990,
    max=2020,
    step=5,
    marks={
        1990: '1990',
        1995: '1995',
        2000: '2000',
        2005: '2005',
        2010: '2010',
        2015: '2015',
        2020: '2020'
    },
    value=1995  # Default year
)

slider_refugees = dcc.Slider(
    id='year-slider-refugees',
    min=1990,
    max=2020,
    step=5,
    marks={
        1990: '1990',
        1995: '1995',
        2000: '2000',
        2005: '2005',
        2010: '2010',
        2015: '2015',
        2020: '2020'
    },
    value=1995  # Default year
)

slider_immigrants = dcc.Slider(
    id='year-slider-immigrants',
    min=1990,
    max=2020,
    step=5,
    marks={
        1990: '1990',
        1995: '1995',
        2000: '2000',
        2005: '2005',
        2010: '2010',
        2015: '2015',
        2020: '2020'
    },
    value=1995  # Default year
)

slider_net_migration = dcc.Slider(
    id='year-slider-net-migration',  # Changed to match the callback input
    min=1990,
    max=2020,
    step=5,
    marks={
        1990: '1990',
        1995: '1995',
        2000: '2000',
        2005: '2005',
        2010: '2010',
        2015: '2015',
        2020: '2020'
    },
    value=1995  # Default year
)

# Choropleth maps and DataTables placeholders
graph_emigrants = dcc.Graph(id='emigrant-map')
graph_immigrants = dcc.Graph(id='immigrant-map')
graph_net_migration = dcc.Graph(id='net-migration-map')
graph_refugees = dcc.Graph(id='refugees-map')

# Layout for both emigrants and immigrants
app.layout = html.Div([
    html.H1("Global Migration Data Visualization"),

    html.Div([
        html.H2("Emigrants by Country"),
        html.Div([
            dcc.Graph(id='emigrant-map', style={'flex': '1'}),  # Emigrant choropleth map
            dcc.Graph(id='emigrant-bar', style={'flex': '1', 'padding-left': '20px'}),  # Emigrant bar chart
            html.Div([
                html.H4("Select Year", style={'text-align': 'center', 'margin-bottom': '10px'}),  # Title for the slider
                dcc.Slider(
                    id='year-slider-emigrants',
                    min=1990,
                    max=2020,
                    step=5,
                    marks={year: str(year) for year in range(1990, 2025, 5)},
                    value=1995
                )
            ], style={'flex': '1', 'padding': '10px'})
        ], style={'display': 'flex', 'align-items': 'center'})  # Flex container
    ], style={'margin-bottom': '50px'}),

    # Immigrant Section
    html.Div([
        html.H2("Immigrants by Country"),
        html.Div([
            dcc.Graph(id='immigrant-map', style={'flex': '1'}),  # Immigrant choropleth map
            dcc.Graph(id='immigrant-bar', style={'flex': '1', 'padding-left': '20px'}),  # Immigrant bar chart
            html.Div([
                html.H4("Select Year", style={'text-align': 'center', 'margin-bottom': '10px'}),  # Title for the slider
                dcc.Slider(
                    id='year-slider-immigrants',
                    min=1990,
                    max=2020,
                    step=5,
                    marks={year: str(year) for year in range(1990, 2025, 5)},
                    value=1995
                )
            ], style={'flex': '1', 'padding': '10px'})
        ], style={'display': 'flex', 'align-items': 'center'})  # Flex container
    ], style={'margin-bottom': '50px'}),

    # Net Migration Visualization Section
    html.Div([
        html.H2("Net migration by Country"),
        html.Div([
            dcc.Graph(id='net-migration-map', style={'flex': '1'}),  # Net-migration choropleth map
            dcc.Graph(id='net-migration-bar', style={'flex': '1', 'padding-left': '20px'}),  # Net-migration bar chart
            html.Div([
                html.H4("Select Year", style={'text-align': 'center', 'margin-bottom': '10px'}),  # Title for the slider
                dcc.Slider(
                    id='year-slider-net-migration',
                    min=1990,
                    max=2020,
                    step=5,
                    marks={year: str(year) for year in range(1990, 2025, 5)},
                    value=1995
                )
            ], style={'flex': '1', 'padding': '10px'})
        ], style={'display': 'flex', 'align-items': 'center'})  # Flex container for layout
    ], style={'margin-bottom': '50px'}),

    html.Div([
        html.H2("Refugees by Country"),
        html.Div([
            dcc.Graph(id='refugees-map', style={'flex': '1'}),  # Refugees choropleth map
            dcc.Graph(id='refugees-bar', style={'flex': '1', 'padding-left': '20px'}),  # Refugees bar chart
            html.Div([
                html.H4("Select Year", style={'text-align': 'center', 'margin-bottom': '10px'}),  # Title for the slider
                dcc.Slider(
                    id='year-slider-refugees',
                    min=1990,
                    max=2020,
                    step=5,
                    marks={year: str(year) for year in range(1990, 2025, 5)},
                    value=1995
                )
            ], style={'flex': '1', 'padding': '10px'})
        ], style={'display': 'flex', 'align-items': 'center'})  # Flex container for layout
    ], style={'margin-bottom': '50px'})

])

## Dash Application for Global Migration Data Visualization

This Dash application visualizes migration data through choropleth maps and bar charts for different migration indicators such as emigrants, immigrants, net migration, and refugees. The data is displayed for the years of interest (1990-2020), with users able to select the year using sliders.

### Key Components of the Layout

1. **App Structure**:
   The application layout is divided into four main sections:
   - **Emigrants by Country**
   - **Immigrants by Country**
   - **Net Migration by Country**
   - **Refugees by Country**

2. **Sliders for Year Selection**:
   Each section includes a slider that allows the user to select a year from 1990 to 2020. The slider values range from 1990 to 2020, with a step of 5 years. The selected year will update the displayed data on the corresponding choropleth map and bar chart.

   Each slider has the following parameters:
   - `id`: Unique identifier for the slider.
   - `min`: Minimum value of the slider (1990).
   - `max`: Maximum value of the slider (2020).
   - `step`: Step size (5).
   - `marks`: Labels for each year, ranging from 1990 to 2020.
   - `value`: Default selected year (1995).

3. **Choropleth Maps and Bar Charts**:
   Each section features two visualizations:
   - A **Choropleth Map** to visualize the data geographically.
   - A **Bar Chart** to visualize the data for individual countries.

4. **Layout Design**:
   - The layout uses a `flex` display to arrange elements in rows with choropleth maps and bar charts side-by-side.
   - Each section has a header (`H2`) for the title and a `H4` for the year selection prompt.

5. **HTML Structure**:
   The application uses `html.Div` to structure the layout. Each section is wrapped in a `Div` with appropriate styling for spacing (`margin-bottom`) and alignment.


In [51]:

# Callback for emigrant visualizations
@app.callback(
    [Output('emigrant-map', 'figure'),
     Output('emigrant-bar', 'figure')],
    [Input('year-slider-emigrants', 'value')]
)
def update_emigrant_visualizations(selected_year):
    data = {
        'country': [],
        'value': []
    }
    for country in country_names_geojson:
        if country in country_data and selected_year in country_data[country]:
            data['country'].append(country)
            data['value'].append(country_data[country][selected_year]["emigrants"])
        else:
            data['country'].append(country)
            data['value'].append(0)

    df = pd.DataFrame(data)
    df_sorted = df.sort_values(by='value', ascending=False)

    # Choropleth map
    fig_map = px.choropleth(df, locations='country', locationmode='country names', color='value',
                            hover_name='country', color_continuous_scale='Viridis',
                            title=f"Total Emigrants by Country ({selected_year})")

    # Bar chart for top 10 countries
    fig_bar = px.bar(
        df_sorted.head(10),
        x='value',
        y='country',
        orientation='h',
        color='value',
        labels={'value': 'Number of Emigrants', 'country': 'Country'},
        title=f"Top 10 Emigrant Countries in {selected_year}",
        color_continuous_scale='Viridis'
    )
    fig_bar.update_layout(
        yaxis=dict(autorange="reversed"),
        showlegend=False
    )

    return fig_map, fig_bar


# Callback for immigrant visualizations
@app.callback(
    [Output('immigrant-map', 'figure'),
     Output('immigrant-bar', 'figure')],
    [Input('year-slider-immigrants', 'value')]
)
def update_immigrant_visualizations(selected_year):
    data = {
        'country': [],
        'value': []
    }
    for country in country_names_geojson:
        if country in country_data and selected_year in country_data[country]:
            data['country'].append(country)
            data['value'].append(country_data[country][selected_year]["immigrants"])
        else:
            data['country'].append(country)
            data['value'].append(0)

    df = pd.DataFrame(data)
    df_sorted = df.sort_values(by='value', ascending=False)

    # Choropleth map
    fig_map = px.choropleth(df, locations='country', locationmode='country names', color='value',
                            hover_name='country', color_continuous_scale='Plasma',
                            title=f"Total Immigrants by Country ({selected_year})")

    # Bar chart for top 10 countries
    fig_bar = px.bar(
        df_sorted.head(10),
        x='value',
        y='country',
        orientation='h',
        color='value',
        labels={'value': 'Number of Immigrants', 'country': 'Country'},
        title=f"Top 10 Immigrant Countries in {selected_year}",
        color_continuous_scale='Plasma'
    )
    fig_bar.update_layout(
        yaxis=dict(autorange="reversed"),
        showlegend=False
    )

    return fig_map, fig_bar


# Callback for net-migration visualizations
@app.callback(
    [Output('net-migration-map', 'figure'),
     Output('net-migration-bar', 'figure')],
    [Input('year-slider-net-migration', 'value')]
)
def update_immigrant_visualizations(selected_year):
    data = {
        'country': [],
        'value': []
    }
    for country in country_names_geojson:
        if country in country_data and selected_year in country_data[country]:
            emigrants = country_data[country][selected_year]["emigrants"]
            immigrants = country_data[country][selected_year]["immigrants"]
            net_migration = immigrants - emigrants  # Calculate net migration

            data['country'].append(country)
            data['value'].append(net_migration)
        else:
            data['country'].append(country)
            data['value'].append(0)

    df = pd.DataFrame(data)
    df_sorted = df.sort_values(by='value', ascending=False)

    # Choropleth map for net migration (positive or negative)
    fig_map = px.choropleth(df, locations='country', locationmode='country names', color='value',
                            hover_name='country', color_continuous_scale='RdBu',  # Red-Blue color scale to show negative and positive values
                            title=f"Total Net Migration by Country ({selected_year})")

    # Bar chart for top 10 countries
    fig_bar = px.bar(
        df_sorted.head(10),
        x='value',
        y='country',
        orientation='h',
        color='value',
        labels={'value': 'Number of Net Migration', 'country': 'Country'},
        title=f"Top 10 Countries by Net Migration in {selected_year}",
        color_continuous_scale='RdBu'  # Red-Blue color scale for better contrast
    )
    fig_bar.update_layout(
        yaxis=dict(autorange="reversed"),
        showlegend=False
    )

    return fig_map, fig_bar

# Callback for refugee visualizations
@app.callback(
    [Output('refugees-map', 'figure'),
     Output('refugees-bar', 'figure')],
    [Input('year-slider-refugees', 'value')]
)
def update_refugee_visualizations(selected_year):
    data = {
        'country': [],
        'value': []
    }
    for country in country_names_geojson:
        if country in refugees_dict and selected_year in refugees_dict[country]:
            refugee_count = refugees_dict[country][selected_year]

            data['country'].append(country)
            data['value'].append(refugee_count)
        else:
            data['country'].append(country)
            data['value'].append(0)

    df = pd.DataFrame(data)
    df_sorted = df.sort_values(by='value', ascending=False)

    # Choropleth map for refugees
    fig_map = px.choropleth(df, locations='country', locationmode='country names', color='value',
                            hover_name='country', color_continuous_scale='YlGnBu',  # Yellow-Green-Blue color scale
                            title=f"Total Refugees by Country ({selected_year})")

    # Bar chart for top 10 countries by refugees
    fig_bar = px.bar(
        df_sorted.head(10),
        x='value',
        y='country',
        orientation='h',
        color='value',
        labels={'value': 'Number of Refugees', 'country': 'Country'},
        title=f"Top 10 Countries by Refugees in {selected_year}",
        color_continuous_scale='YlGnBu'  # Green-Blue color scale
    )
    fig_bar.update_layout(
        yaxis=dict(autorange="reversed"),
        showlegend=False
    )

    return fig_map, fig_bar


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

# Migration Data Visualization App

This project is a web application built with Dash that visualizes migration data across four categories: Emigrants, Immigrants, Net Migration, and Refugees. The data is displayed dynamically using choropleth maps and bar charts. Users can interact with the visualizations by selecting different years from the provided sliders.

## Features
- **Interactive Visualization**: Explore migration trends over time with interactive choropleth maps and bar charts.
- **Dynamic Year Selection**: Choose different years (1990-2020) using sliders to update the data.
- **Multiple Migration Categories**: Visualize data for Emigrants, Immigrants, Net Migration, and Refugees.

## How it Works

### App Setup
The app is built using the **Dash** framework. It consists of the following components:
1. **Sliders**: There are four sliders, each controlling the year selection for a specific category (Emigrants, Immigrants, Net Migration, Refugees).
   - The sliders range from 1990 to 2020, with increments of 5 years.
   
2. **Choropleth Maps**: For each migration category, there is a choropleth map showing the migration data by country for the selected year.
   
3. **Bar Charts**: Bar charts display the top 10 countries for each migration category, ordered by the number of emigrants, immigrants, net migration, or refugees.

### Callbacks
The app uses **Dash callbacks** to update the visualizations based on the selected year from the sliders. The callbacks are structured as follows:

#### 1. **Emigrant Visualization**
   - **Input**: `year-slider-emigrants`
   - **Output**: Choropleth map and bar chart showing emigrant data for the selected year.
   - **Logic**:
     - The data for each country and year is fetched from `country_data`.
     - The map visualizes the total number of emigrants by country using Plotly's choropleth map.
     - The bar chart displays the top 10 countries with the highest emigrant numbers.

#### 2. **Immigrant Visualization**
   - **Input**: `year-slider-immigrants`
   - **Output**: Choropleth map and bar chart showing immigrant data for the selected year.
   - **Logic**:
     - Similar to the emigrant visualization, the data is fetched from `country_data` for the selected year.
     - The map and bar chart visualize the total number of immigrants by country.

#### 3. **Net Migration Visualization**
   - **Input**: `year-slider-net-migration`
   - **Output**: Choropleth map and bar chart showing net migration data (immigrants - emigrants) for the selected year.
   - **Logic**:
     - The net migration for each country is calculated as the difference between the number of immigrants and emigrants.
     - A choropleth map and bar chart are generated to show the net migration by country.

#### 4. **Refugee Visualization**
   - **Input**: `year-slider-refugees`
   - **Output**: Choropleth map and bar chart showing refugee data for the selected year.
   - **Logic**:
     - The refugee data for each country is fetched from `refugees_dict`.
     - A choropleth map and bar chart are displayed to visualize the total number of refugees by country.

### Color Scales
- **Emigrants**: `Viridis`
- **Immigrants**: `Plasma`
- **Net Migration**: `RdBu` (Red-Blue) to represent positive and negative values
- **Refugees**: `YlGnBu` (Yellow-Green-Blue)

